# Question 1

On raisonne de manière infinitésimale en notant $Q_j(t)$ la charge du véhicule $j$ au temps $t$. Lorsque le véhicule est en charge, la station fournit au véhicule $j$ une puissance $P_j(t)$, et la voiture est capable de capter une certaine fraction $\beta_j$ de cette puissance ($\beta_j$ sera proche de 1 pour une batterie neuve, et tendra vers 0 quand la batterie s'use). Ainsi, la dérivée de la charge du véhicule (en terme de dimension, $Q_j$ sera une énergie) vaut $\frac{dQ_j(t)}{dt} = \beta_j P_j(t)$.

Si partitionne notre intervalle de temps en N intervalles $[t_i, t{i+1}]$ de durée $\Delta t$ (qui est donc la plus petite durée considérée), cette équation s'approxime par $\frac{Q_j(t + \Delta t) - Q_j(t)}{\Delta t} = \beta_j P_j(t)$ ou encore $\frac{Q_{j, i+1} - Q_{j, i}}{\Delta t} = \beta_j P_{j,i}$ i.e :
$$
Q_{j, i+1} =  Q_{j, i} + \Delta t \beta_j P_{j,i}
$$